This notebook provides template functions to initialize a DuckDB database from default and split CSV files. Subsequent data analysis in Ibis can proceed agnostic to the underlying format.

In [1]:
def init_ddb_from_csv(db_filename, tablename, csv_filename, **kwargs):
    """
    Load from the csv file into a DuckDB database.
    
    db_filename: Name of the database
    tablename: Table to load to
    csv_filename: CSV file to load from
    **kwargs: Options for DuckDB's read_csv function, see https://duckdb.org/docs/data/csv/overview
    """
    import duckdb
    duckdb_con = duckdb.connect(db_filename)
    read_csv_args_list = ["'{}'".format(csv_filename)]
    for key, value in kwargs.items():
        read_csv_args_list.append("{0} = {1}".format(key, value))
    read_csv_args = ','.join(read_csv_args_list)
    sql_stmt = "CREATE TABLE {} AS SELECT * FROM read_csv({}, AUTO_DETECT=TRUE)".format(tablename, read_csv_args)
    print(sql_stmt)
    duckdb_con.sql(sql_stmt)
    duckdb_con.close()

def init_ddb_from_split_csv(db_filename, tablename, split_csv_foldername, **kwargs):
    """
    Load the split csv file into a DuckDB database and expose a view with tablename

    db_filename: Name of the database
    tablename: View to expose giving the impression of a table
    csv_filename: Folder containing the split CSV files
    **kwargs: Options for DuckDB's read_csv function, see https://duckdb.org/docs/data/csv/overview
    """
    import duckdb
    import os
    duckdb_con = duckdb.connect(db_filename)
    # read_csv_args_list = ["'{}'".format(csv_filename)]
    # for key, value in kwargs.items():
    #     read_csv_args_list.append("{0} = {1}".format(key, value))
    # read_csv_args = ','.join(read_csv_args_list)
    num_dims = 0
    cols = []
    sub_tablenames = []
    for root, dirs, files in os.walk(split_csv_foldername):
        for file in files:
            if 'dim' in file:
                num_dims += 1
            sub_tablename = tablename + "_" + file.split(".csv")[0]
            sub_tablenames.append(sub_tablename)
            full_filename = root + '/' + file
            sql_stmt = "CREATE TABLE {} AS SELECT * FROM read_csv('{}', AUTO_DETECT=TRUE)".format(sub_tablename, full_filename)
            print(sql_stmt)
            duckdb_con.sql(sql_stmt)
            table = duckdb_con.table(sub_tablename)
            for col in table.columns:
                # HACK: not fool proof, the CSV could contain a column starting with letter 'p'
                if col[0] == 'p':
                    continue
                cols.append('"' + col + '"')

    # Now create a view corresponding to a single original csv file
    join_clauses = []
    for i in range(num_dims):
        join_clause = "{}_fact.p{} = {}_dim{}.p{}".format(tablename, i, tablename, i, i)
        join_clauses.append(join_clause)

    sql_stmt = "CREATE VIEW {} AS SELECT ".format(tablename) + ",".join(cols) + \
        " FROM " + ",".join(sub_tablenames) + " WHERE " + (" AND ").join(join_clauses)
    print(sql_stmt)
    duckdb_con.sql(sql_stmt)
    duckdb_con.close()

In [2]:
init_ddb_from_csv("us-accidents/us_accidents_default.db", "accidents", "us-accidents/US_Accidents_Dec21_updated.csv")

CREATE TABLE accidents AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated.csv', AUTO_DETECT=TRUE)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
# Cleanup
!rm us-accidents/us_accidents_default.db*

In [4]:
init_ddb_from_split_csv("us-accidents/us_accidents_split.db", "accidents", "us-accidents/US_Accidents_Dec21_updated_split_csv") 

CREATE TABLE accidents_dim8 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim8.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim7 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim7.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim1 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim1.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim9 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim9.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim3 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim3.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim10 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim10.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim5 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim5.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim11 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CREATE TABLE accidents_dim6 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim6.csv', AUTO_DETECT=TRUE)
CREATE TABLE accidents_dim2 AS SELECT * FROM read_csv('us-accidents/US_Accidents_Dec21_updated_split/dim2.csv', AUTO_DETECT=TRUE)
CREATE VIEW accidents AS SELECT "Distance(mi)","City","Severity","Timezone","Wind_Direction","State","Street","Wind_Speed(mph)","Precipitation(in)","Zipcode","Pressure(in)","Weather_Timestamp","Turning_Loop","Country","Traffic_Signal","Traffic_Calming","Stop","Station","Roundabout","Railway","No_Exit","Junction","Give_Way","Crossing","Bump","Amenity","Astronomical_Twilight","Nautical_Twilight","Civil_Twilight","Sunrise_Sunset","Side","Description","Temperature(F)","Wind_Chill(F)","ID","Start_Time","End_Time","Start_Lat","Start_Lng","End_Lat","End_Lng","Number","Airport_Code","County","Visibility(mi)","Humidity(%)","Weather_Condition" FROM accidents_dim8,accidents_dim7,accidents_dim1,accidents_dim9,accidents_dim3,accidents_dim10,ac

In [5]:
# Cleanup
!rm us-accidents/us_accidents_split.db*